## Packages

In [1]:
import pandas as pd             # Traitement des tableaux de données 

whole_data = pd.read_csv('Data/TRAIN_FULL/BAAC_2012_full.csv')

# Pré-Traitement des données

In [2]:
whole_data.head()

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,...,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1
0,201200000001,1.0,1,1,2,5.0,11.0,0.0,0.0,0.0,...,875.0,0.0,1.0,1.0,0.0,72.0,1.0,0.0,1.0,0.0
1,201200000001,NaN,3,4,1,5.0,NaN,3.0,3.0,2.0,...,875.0,0.0,1.0,1.0,0.0,72.0,1.0,0.0,1.0,0.0
2,201200000002,1.0,1,1,1,5.0,13.0,0.0,0.0,0.0,...,250.0,0.0,1.0,1.0,0.0,65.0,1.0,0.0,1.0,99.0
3,201200000002,2.0,2,1,1,0.0,11.0,0.0,0.0,0.0,...,250.0,0.0,1.0,1.0,0.0,65.0,1.0,0.0,1.0,99.0
4,201200000002,1.0,1,1,1,5.0,21.0,0.0,0.0,0.0,...,250.0,0.0,1.0,1.0,0.0,65.0,1.0,0.0,1.0,99.0


### Num_Acc

Identifiant de l'accident. Valeur vitale pour joindre les données. Cependant, au-delà de la fusion des données, ce numéro n'apporte rien en matière de prédiction. En effet, il est générique et ne transmet pas d'information complémentaire. On se demande alors s'il ne vaudrait pas mieux le supprimer.

In [359]:
print(f"Le nombre d'observations avec une absence d'identifiant d'accidant est de : {whole_data['Num_Acc'].isna().sum()}.")
print(f"Le nombre d'observations unique d'identifiant d'accidant est de : {whole_data['Num_Acc'].nunique()}.")

whole_data.drop('Num_Acc', axis=1, inplace=True)

Le nombre d'observations avec une absence d'identifiant d'accidant est de : 0.
Le nombre d'observations unique d'identifiant d'accidant est de : 56025.


### place

Permet de situer la place occupée dans le véhicule par l'usager au moment de l'accident. Les informations concernants les positions sont trouvables au sein du fichier **"*description-des-bases-de-donnees-onisr-annees-2005-a-2020.pdf*"** notamment. Cette donnée est majeure car il existe certainement une relation entre la position des victimes et la gravité de l'accident. En effet, en fonction de la position, les dispositifs de sécurité ne sont pas les mêmes. On soupçonne donc une importante majeure.

In [341]:
possible_values = whole_data['place'].unique()
na_count = whole_data['place'].isna().sum()
obs_count = whole_data.shape[0]
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [ 1. nan  2.  3.  4.  8.  5.  6.  7.  9.].
Le nombre de valeurs manquantes est de : 11174 pour un total de 124598 observations soit 8.968041220565338%.


Etant donné qu'il est difficile de prédire une position manquante car cela dépend notamment du véhicule, il est donc décidé de supprimer les lignes manquantes. Cependant, il est nécessaire d'attendre la prochaine variable afin d'ajouter une condition. En effet, selon la documentation, **'*10*'** doit correspondre au piéton mais il n'apparaît pas. Ainsi, seules les lignes contenant une valeur **"*Nan*"** **et** qui ne matcheront pas la position de piéton seront supprimées.

### catu

Correspond à la catégorie d'usager. Autrement dit, s'il s'agit d'un conducteur, d'un passager ou bien d'un piéton. Il est important de disposer de ces informations en cas de corrélations entre le statut de la victime et son état. On peut soumettre l'hypothèse que ce statut influe sur les situations dans lesquelles les usagers sont impliqués notamment (difficile d'aller à 130 km/h en tant que piéton) :

In [360]:
# Supprime seulement si 'place' est Nan ET que ce n'est pas un piéton
whole_data = whole_data[~((whole_data['catu'] != 3) & whole_data['place'].isna())]

possible_values = whole_data['catu'].unique()
na_count = whole_data['catu'].isna().sum()
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [1 3 2].
Le nombre de valeurs manquantes est de : 0 pour un total de 113424 observations soit 0.0%.


### grav

Etant la colonne à prédire, il est évident que nous souhaitons conserver cette dernière. Nous nous penchons sur la qualité de cette dernière :

In [361]:
possible_values = whole_data['grav'].unique()
na_count = whole_data['grav'].isna().sum()
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [1 4 3 2].
Le nombre de valeurs manquantes est de : 0 pour un total de 113424 observations soit 0.0%.


Tout semble en bon état, les valeurs possibles respectent la documentation. Cependant, dans l'exercice de prédiction, il est demandé de convertir cette colonne en binaire (**'*0*'** pour les cas sans dangers [1, 4] et **'*1*'** pour les cas graves [2, 3]) : 

In [362]:
whole_data['grav'] = whole_data['grav'].replace({1: 0, 4: 0, 2: 1, 3: 1})

possible_values = whole_data['grav'].unique()

print(f"Les valeurs possibles sont : {possible_values}.")

Les valeurs possibles sont : [0 1].


### sexe

Certaines études ont pu démontrer des disparités dans les comportements de conduite entre les différents sexes. Nous conservons cette donnée dans l'éventualité où une corrélation existerait entre le sexe, le statut de conducteur et la gravité :

In [364]:
possible_values = whole_data['sexe'].unique()
na_count = whole_data['sexe'].isna().sum()
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [2 1].
Le nombre de valeurs manquantes est de : 0 pour un total de 113424 observations soit 0.0%.


### trajet